#### Importing libraries

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
from zipfile import ZipFile  #To unzip data
import cv2
import matplotlib.pyplot as plt

In [ ]:
#Importing library from tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1

#### Read train , test csv as data frame

Read csv and add a new coloumn with .jpg appended in id( unique image number)

In [ ]:
df_train = pd.read_csv('path/train_data.csv',error_bad_lines=False)
df_train['image'] = df_train.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [ ]:
df_test = pd.read_csv('path/test_data.csv',error_bad_lines=False)
df_test['image'] = df_test.apply(lambda row: str(row['id']) + ".jpg", axis=1)

#### Use Image Data genrator to read images in batch 
#### Use Augumantion & class weight 

In [ ]:
# Data augmentation with different settings
image_generator = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=15,   # Random rotation
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.1,  # Shear transformation
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)
####################################
# count of each class
class_counts = df_test['articleType'].value_counts().to_dict()
#####################################
# Class weights for balancing
class_weights = {class_idx: max(class_counts.values()) / count for class_idx, count in class_counts.items()}

In [ ]:
# Flow training data from DataFrame with data augmentation
training_generator = image_generator.flow_from_dataframe(
    dataframe=df_train,
    directory='path/train_image',
    x_col="image",
    y_col="articleType",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical", 
    shuffle=True,
    class_weights=class_weights  
)

In [ ]:
df_test

In [ ]:
# validation data from DataFrame without augmentation
validation_generator = image_generator.flow_from_dataframe(
    dataframe=df_test,
    directory="path/test_image",
    x_col="image",
    y_col="articleType",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical",  # Use "categorical" for one-hot encoded labels
    shuffle=False
)

#### Define model

In [ ]:
# Sequential model
model_1= Sequential()

# ResNet50 model
model_1.add(ResNet50(include_top=False, weights="imagenet", pooling="avg"))

# Adding new layers
model_1.add(Dense(128, activation="relu"))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))

model_1.add(Dense(64, activation="relu",kernel_regularizer=l1(0.001)))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))

model_1.add(Dense(32, activation="relu",kernel_regularizer=l1(0.001)))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.4))
model_1.add(Dense(18, activation="softmax"))

#### Complie the model

In [ ]:
# adding Early Stopping and Model Checkpoint
early_stopping = EarlyStopping(monitor="val_acc", patience=2, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.h5", monitor="val_acc", save_best_only=True)
##
optimizer_ = Adam(learning_rate=0.001)
# Compile the model
model_1.compile(optimizer=optimizer_, loss="categorical_crossentropy", metrics=["accuracy"])

#### Train the model

In [ ]:
# Train the model
history_1= model_1.fit(training_generator,steps_per_epoch=len(training_generator),epochs=17,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping, model_checkpoint]
)